In [1]:
import os
import json
import requests
import geojsonio
import time
import geopandas as gpd
import geojson
from os.path import dirname as up
import pandas as pd

import json
import requests
from requests.auth import HTTPBasicAuth

from tqdm import tqdm

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))



In [2]:
root_path = up(os.getcwd())
meta_path = os.path.join(root_path, 'data/processing_data/planetMetadata/planetMeta_2022JanToJun_Kory_edit.csv')
dataDir = os.path.join(root_path, 'data/output')
df = pd.read_csv(meta_path)

In [3]:
df = df[df.Useful_Scene=='Y']

In [4]:
df

,acquired,anomalous_pixels,clear_confidence_percent,clear_percent,cloud_cover,cloud_percent,ground_control,gsd,heavy_haze_percent,instrument,...,sun_azimuth,sun_elevation,updated,view_angle,visible_confidence_percent,visible_percent,id,date,hour_min_sec,Useful_Scene
1,2022-01-08T14:57:24.181709Z,0,97,100,0,0,True,4.0,0,PSB.SD,...,146.6,22.9,2022-10-23T23:50:13Z,5.0,74,100,20220108_145724_18_2431,20220108,145724,Y
4,2022-01-10T15:47:41.228478Z,0,99,100,0,0,True,4.0,0,PSB.SD,...,157.9,27.6,2022-10-24T10:30:49Z,5.0,99,100,20220110_154741_22_2405,20220110,154741,Y
7,2022-01-11T15:01:33.411001Z,0,98,100,0,0,True,4.0,0,PSB.SD,...,146.3,23.2,2022-10-24T08:00:23Z,3.0,98,100,20220111_150133_41_2465,20220111,150133,Y
28,2022-02-10T15:50:59.333808Z,0,99,100,0,0,True,4.0,0,PSB.SD,...,152.9,34.1,2023-05-13T23:49:35Z,5.1,99,100,20220210_155059_33_2407,20220210,155059,Y
31,2022-02-12T14:57:42.056045Z,0,98,100,0,0,True,4.1,0,PSB.SD,...,140.1,29.5,2023-05-14T07:54:10Z,5.0,98,100,20220212_145742_05_2421,20220212,145742,Y
52,2022-03-08T14:59:18.168936Z,0,100,100,0,0,True,4.1,0,PSB.SD,...,135.1,37.3,2023-01-28T06:00:47Z,1.0,81,100,20220308_145918_16_2458,20220308,145918,Y
55,2022-03-13T14:57:05.357325Z,0,99,100,0,0,True,4.1,0,PSB.SD,...,134.0,39.1,2023-01-30T09:30:40Z,5.1,99,100,20220313_145705_35_242d,20220313,145705,Y
66,2022-03-29T14:56:08.144131Z,0,97,100,0,0,True,4.0,0,PSB.SD,...,129.5,44.3,2023-01-27T05:30:18Z,2.0,97,100,20220329_145608_14_2449,20220329,145608,Y
78,2022-04-11T14:56:29.267547Z,0,100,100,0,0,True,4.0,0,PSB.SD,...,126.0,48.6,2022-04-13T12:59:41Z,1.0,100,100,20220411_145629_26_2447,20220411,145629,Y
82,2022-04-13T14:57:46.689551Z,0,98,99,0,0,True,4.0,0,PSB.SD,...,125.5,49.3,2022-04-14T17:02:04Z,5.0,67,100,20220413_145746_68_2457,20220413,145746,Y


In [5]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = ''

# construct auth tuple for use in the requests library
BASIC_AUTH = (API_KEY, '')

# Setup Planet Data API base URL
URL = "https://api.planet.com/data/v1"

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

# Make a GET request to the Planet Data API
res = session.get(URL)

# Response status code
res.status_code

200

In [6]:
def activating_asset(sid, itemType, productKey):
    
    id_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(itemType, sid)
    # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
    result = \
      requests.get(
        id_url,
        auth=HTTPBasicAuth(API_KEY, '')
      )

    # Parse out useful links
    links = result.json()[productKey]["_links"]
    self_link = links["_self"]
    activation_link = links["activate"]

    # Request activation of the asset:
    activate_result = \
      requests.get(
        activation_link,
        auth=HTTPBasicAuth(API_KEY, '')
      )
    
    activation_status_result = \
      requests.get(
        self_link,
        auth=HTTPBasicAuth(API_KEY, '')
      )

    return activation_status_result.json()["status"]



# Create a function to download asset files
# Parameters: 
# - url (the location url)
# - filename (the filename to save it as. defaults to whatever the file is called originally)

def pl_download(sid, itemType, productKey, downloadDir, filename=None):
    
    assets_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets/'.format(itemType, sid)
    res = session.get(assets_url)
    assets = res.json()
    dataproduct = assets[productKey]
    location_url = dataproduct["location"]

    # Send a GET request to the provided location url, using your API Key for authentication
    res = requests.get(location_url, stream=True, auth=(API_KEY, ""))
    # If no filename argument is given
    if not filename:
        # Construct a filename from the API response
        if "content-disposition" in res.headers:
            filename = res.headers["content-disposition"].split("filename=")[-1].strip("'\"")
        # Construct a filename from the location url
        else:
            filename = location_url.split("=")[1][:10]
    # Save the file
    if os.path.isfile(os.path.join(downloadDir, filename)):
        return 'File {} exists in the directory'.format(filename)
    else:
        with open(os.path.join(downloadDir, filename), "wb") as f:
            for chunk in res.iter_content(chunk_size=1024):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    f.flush()

        return filename

# Download the file from an activated asset's location url
# pl_download(location_url)

In [9]:
allids = df.id.tolist()
item_type = "PSScene"
instruKey = "ortho_analytic_8b"

# for sID in tqdm(allids): 
    
#     id_status = activating_asset(sID, item_type, instruKey)
#     if id_status == 'active':
#         pl_download(sID, item_type, instruKey, dataDir, filename=None)
#     else:
#         print('Scene id {} is not active'.format(sID))
    

100%|██████████| 18/18 [06:18<00:00, 21.05s/it]
